# Notebook para procesar datos de AWS Application Discovery Service

## 1. Definición de Parámetros

Este notebook sirve para realizar la unión y el procesamiento de datos que entrega a la salida el servicio de AWS ADS. Podemos realizar la conjunción de los archivos:

1. EC2 Instance Recommendations
2. System Performance

La información que obtenemos a la salida de este Jupyter Notebook es:

1. Estimación de costos de EC2
2. Estimación de costos de almacenamiento EBS
3. Estimación de DTO, considerando que un 20% de tráfico escrito a la red saldrá a Internet

Adicionalmente, obtenemos CSVs con los siguientes datos:

1. Instancias recomendadas, con datos de Almacenamiento, DTO, RAM, IOPs y vCPUs por cada instancia
2. VMs que no se pudieron procesar
3. Instancias que no tienen actividad
4. Instancias con menos de 10 IOPs
5. Las 10 instancias más costosas

Favor de modificar los datos siguientes, de acuerdo los datos de su export:

In [ ]:
# Celda 1
# Importando las librerias necesarias
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import boto3
import json


# favor de reemplazar con el nombre de los archivos que tengan a la salida de AWS ADS
ec2_recommendation_file = "EC2InstanceRecommendations.csv"
system_performance_file = "SystemPerformance.csv"


En esta sección se consultan de manera automática los precios de EBS ( fuente: https://aws.amazon.com/ebs/pricing/ ). Favor de cambiar la región si es necesario:

In [ ]:
# Celda 2
print("Get EBS Volume Prices")
print("==============================")

pricing = boto3.client('pricing')

#cambiar abajo 'location' en caso de hacer un TCO en otra región
response = pricing.get_products(
     ServiceCode='AmazonEC2',
     Filters = [
         {'Type' :'TERM_MATCH', 'Field':'volumeApiName', 'Value':'standard'              }
         ,{'Type' :'TERM_MATCH', 'Field':'locationType',  'Value':'AWS Region'                   }
         ,{'Type' :'TERM_MATCH', 'Field':'location',      'Value':'US East (N. Virginia)'              }
     ],
     MaxResults=100
)


# From the whole response get the list of key names (product codes) that we are intereste on,
# which are the OnDemand Price Dimensions. Each range of price has a unique code name that we
# don't know in advance. That's why we have to get the keys list to iterate them later.
product = json.loads(response['PriceList'][0])
ondem = product['terms']['OnDemand']
ondem_list = list(ondem.keys())
prices = ondem[ondem_list[0]]['priceDimensions']
prices_list = list(prices.keys())

# Based on the top GBs of the range determine the cost variable of interest
for range_code in prices_list:
    range = prices[range_code] 
    if range['endRange'] == 'Inf':
        ebs_magnetic_monthly_cost  = float(range['pricePerUnit']['USD'])

print('ebs_magnetic_monthly_cost =  $ {} USD'.format(ebs_magnetic_monthly_cost))

##########################################

#cambiar abajo 'location' en caso de hacer un TCO en otra región
response = pricing.get_products(
     ServiceCode='AmazonEC2',
     Filters = [
         {'Type' :'TERM_MATCH', 'Field':'volumeApiName', 'Value':'gp3'              }
         ,{'Type' :'TERM_MATCH', 'Field':'locationType',  'Value':'AWS Region'                   }
         ,{'Type' :'TERM_MATCH', 'Field':'location',      'Value':'US East (N. Virginia)'              }
     ],
     MaxResults=100
)

# From the whole response we search for the specific "usagetype" we are looking for between all prices.
# From thta product we get the list of key names (product codes) that we are intereste on,
# which are the OnDemand Price Dimensions. Each range of price has a unique code name that we
# don't know in advance. That's why we have to get the keys list to iterate them later.
pricelist = response['PriceList']
for i in pricelist:
    tmp_prod = json.loads(i)
    if tmp_prod['product']['attributes']['usagetype'] == 'EBS:VolumeUsage.gp3':
        product = tmp_prod

ondem = product['terms']['OnDemand']
ondem_list = list(ondem.keys())
prices = ondem[ondem_list[0]]['priceDimensions']
prices_list = list(prices.keys())

# Based on the top GBs of the range determine the cost variable of interest
for range_code in prices_list:
    range = prices[range_code]
    if range['endRange'] == 'Inf':
        ebs_gp3_monthly_cost   = float(range['pricePerUnit']['USD'])

print('ebs_gp3_monthly_cost  = $ {} USD'.format(ebs_gp3_monthly_cost ))

En esta sección se consultan de manera automática los precios de Data Transfer Out (DTO) por GiB ( fuente: https://aws.amazon.com/ec2/pricing/on-demand/ ). Favor de cambiar la región si es necesario:

In [ ]:
# Celda 3
print("Get DataTransfer Prices")
print("==============================")

#cambiar abajo 'location' en caso de hacer un TCO en otra región
response = pricing.get_products(
     ServiceCode='AWSDataTransfer',
     Filters = [
         {'Type' :'TERM_MATCH', 'Field':'fromLocationType', 'Value':'AWS Region'              }
         ,{'Type' :'TERM_MATCH', 'Field':'transferType',  'Value':'AWS Outbound'                   }
         ,{'Type' :'TERM_MATCH', 'Field':'fromLocation',      'Value':'US East (N. Virginia)'              }
     ],
     MaxResults=100
)

# From the whole response get the list of key names (product codes) that we are intereste on,
# which are the OnDemand Price Dimensions. Each range of price has a unique code name that we
# don't know in advance. That's why we have to get the keys list to iterate them later.
product = json.loads(response['PriceList'][0])
ondem = product['terms']['OnDemand']
ondem_list = list(ondem.keys())
prices = ondem[ondem_list[0]]['priceDimensions']
prices_list = list(prices.keys())

# Based on the top GBs of the range determine the cost variable of interest
for range_code in prices_list:
    range = prices[range_code]
    
    if range['endRange'] == '10240':
        cost_10_tb = float(range['pricePerUnit']['USD'])
    elif range['endRange'] == '51200':
        cost_40_tb = float(range['pricePerUnit']['USD'])
    elif range['endRange'] == '153600':
        cost_100_tb = float(range['pricePerUnit']['USD'])
    elif range['endRange'] == 'Inf':
        cost_150_tb = float(range['pricePerUnit']['USD'])

print('cost_10_tb = $ {} USD'.format(cost_10_tb))
print('cost_40_tb = $ {} USD'.format(cost_40_tb))
print('cost_100_tb = $ {} USD'.format(cost_100_tb))
print('cost_150_tb = $ {} USD'.format(cost_150_tb))

## 2. Pre-procesamiento de datos

In [ ]:
# Celda 4
# importando el archivo de recomendaciones
ds_rec = pd.read_csv(ec2_recommendation_file)

In [ ]:
# Celda 5
# importando el archivo de performance
ds_sys = pd.read_csv(system_performance_file)

In [ ]:
# Celda 6
# Copiando en un data frame únicamente las columnas de interés del archivo de recomendaciones
ds_rec_fil = ds_rec[['ServerId','Server.HostName','Server.VMware.VMname','Server.OS.Name','Recommendation.EC2.Instance.OSType','Server.VMware.vCenterName','Recommendation.EC2.Instance.vCPUCount','Recommendation.EC2.Instance.RAM.TotalSizeinMB','Recommendation.EC2.Instance.Model','Recommendation.EC2.Instance.Price.HourlyRate','Server.DiskReadOpsPerSecond.Max','Server.DiskWriteOpsPerSecond.Max']].copy()

In [ ]:
# Celda 7
# Renombrando las columnas a algo más fácil de identificar
ds_rec_fil.rename(columns={'ServerId': 'Server_ID','Server.HostName': 'Server','Server.VMware.VMname':'Nombre_en_Vmware','Server.OS.Name':'Operating_System','Recommendation.EC2.Instance.OSType':'Tipo_de_OS','Server.VMware.vCenterName':'Nombre_vCenter','Recommendation.EC2.Instance.vCPUCount':'vCPUs_Recomendacion','Recommendation.EC2.Instance.RAM.TotalSizeinMB':'Memory_(GiB)_Recomendacion','Recommendation.EC2.Instance.Model':'Instancia_EC2_Recomendada','Recommendation.EC2.Instance.Price.HourlyRate':'Precio_por_hr_EC2','Server.DiskReadOpsPerSecond.Max':'DiskReadOpsPerSecondMax','Server.DiskWriteOpsPerSecond.Max':'DiskWriteOpsPerSecondMax'},inplace=True)

In [ ]:
# Celda 8
# Mostrando 10 renglones de manera aleatoria
np.random.seed(1)
ds_rec_fil.sample(n=10)

In [ ]:
# Celda 9
# Copiando las columnas de interés del archivo de performance
ds_sys_fil = ds_sys[['serverId','numDisks','powerState','totalDiskSize','avgNetworkBytesWrittenPerSecond']].copy()

In [ ]:
# Celda 10
# Cambiando el nombre de las columnas a algo más fácil de interpretar
ds_sys_fil.rename(columns={'serverId': 'Server_ID', 'numDisks': 'Num_discos', 'powerState': 'Estado_VM(ON/OFF)','totalDiskSize':'Capacidad_Almacenamiento(GiB)', 'avgNetworkBytesWrittenPerSecond': 'Data_Transfer_Out-GiB_escritos'}, inplace=True)

In [ ]:
# Celda 11
# Mostrando 10 renglones de manera aleatoria
ds_sys_fil.sample(n=10)

In [ ]:
# Celda 12
# Obteniendo el tipo de datos en cada columna del data frame de recomendaciones
ds_rec_fil.info()

In [ ]:
# Celda 13
# Obteniendo el tipo de datos en cada columna del data frame de performance
ds_sys_fil.info()

In [ ]:
# Celda 14
# Juntando ambos data frames (inner join)
df_merged = ds_rec_fil.merge(ds_sys_fil, on='Server_ID', how='inner')

In [ ]:
# Celda 15
# Tomando 10 renglones de manera aleatoria
df_merged.sample(n=10)

In [ ]:
# Celda 16
# Funciones para transformar MiB a GiB y Bytes a GiB

def mb2gb(mb):
    return mb/1024

def b2gb(b):
    if np.isnan(b) == False:
        return round(b/1024/1024/1024)

# Funcion para calcular el consumo de DTO durante un mes, a partir de los bits escritos en red    

def monthly(b):
    if np.isnan(b) == False:
        return round(b*3600*24*30.4)

# Funcion para transfomar bits a GiB

def bi2gb(b):
    if np.isnan(b) == False:
        return round(b/8/1024/1024/1024)

In [ ]:
# Celda 17
# Transformando los datos de Memoria a la escala correcta 
df_merged['Memory_(GiB)_Recomendacion'] = df_merged['Memory_(GiB)_Recomendacion'].apply(mb2gb)

In [ ]:
# Celda 18
# Transformando los datos de DTO a la escala correcta
df_merged['Data_Transfer_Out-GiB_escritos'] = df_merged['Data_Transfer_Out-GiB_escritos'].apply(monthly)
df_merged['Data_Transfer_Out-GiB_escritos'] = df_merged['Data_Transfer_Out-GiB_escritos'].apply(b2gb)

In [ ]:
# Celda 19
# Transformando el almacenamiento a la escala correcta
df_merged['Capacidad_Almacenamiento(GiB)'] = df_merged['Capacidad_Almacenamiento(GiB)'].apply(b2gb)

In [ ]:
# Celda 20
# Tomando 10 renglones de manera aleatoria
df_merged.sample(n=10)

In [ ]:
# Celda 21
df_merged.info()

### Insights:
* Si tenemos una gran cantidad de `Server_ID`, con respecto a `Instancia_EC2_Recomendada`, es posible que ADS no haya recolectado datos de utilización debido a que la máquina está apagada
* No nos interesa hacer la estimación sobre las VMs que están apagadas, vamos a considerar únicamente las instancias cuyo estado sea POWER_ON

In [ ]:
# Celda 22
# Obteniendo sólo las instancias encendidas
df_on = df_merged[df_merged['Estado_VM(ON/OFF)'] == 'POWER_ON']
df_on.info()

### Insights:
* Si aún después de correr el código de arriba, vemos que el número de `Server_ID` non-null > número de `Instancia_EC2_Recomendada` non-null, vamos a copiar los servidores para los cuales AWS ADS no tiene una recomendación
* Hasta cierto punto es normal tener un mayor número de Server_ID a Instancia_EC2_Recomendada, principalmente por:
1. Servidores ESX
2. Aplicaciones de VMware, que no es necesario migrar
3. La aplicación de AWS ADS Connector
4. Instancias que no tienen instaladas VMware Tools

Vamos a exportar todas estas recomendaciones y dejarlas en un CSV para su tratamiento de fuera del Notebook, en caso de ser necesario, utilizando una herramienta como RVTools. Para estos casos, lo recomendable es tratarlos de manera manual, haciendo un direct match a una instancia equivalente EC2

In [ ]:
# Celda 23
# Obteniendo las instancias que no recibieron una recomendación de ADS
df_no_recom = df_on[df_on['Instancia_EC2_Recomendada'].isnull()]

In [ ]:
# Celda 24
# Tomando 5 renglones de manera aleatoria
df_no_recom.sample(n=5)

In [ ]:
# Celda 25
# Guardando las instancias sin recomendacion en un CSV para su análisis fuera del Notebook
df_no_recom.to_csv("EC2SinRecomendacion.csv")

### Removiendo valores Nulos

De manera similar, vamos a filtrar únicamente los valores que no son nulos para hacer el análisis exploratorio de nuestros datos. Con esto podemos realizar una estimación de los costos: EC2, EBS y DTO.

In [ ]:
# Celda 26
# Contando el número de renglones y columnas
df_on_rec = df_on[df_on['Instancia_EC2_Recomendada'].notnull()]

In [ ]:
# Celda 27
df_on_rec.info()

In [ ]:
# Celda 28
# Sumando los IOPs de lectura y de escritura
df_on_rec['Iops_total_(R+W)-Max'] = df_on_rec['DiskReadOpsPerSecondMax'] + df_on_rec['DiskWriteOpsPerSecondMax']


In [ ]:
# Celda 29
df_on_rec.head()

### Optimizando el tipo de volumen EBS

Asumimos que si el número de IOPs es menor a 100, es posible que un volumen de generación previa es suficiente para nuestra aplicación: https://aws.amazon.com/ebs/previous-generation/

Sin embargo, es necesario revisar caso por caso si nuestra aplicación va a funcionar adecuadamente con un volumen magnético.

In [ ]:
# Celda 30
# Dos funciones para determinar el tipo de volumen EBS y su precio, tomando como base los IOPs

def ebs_type(n):
    if n > 100:
        return 'gp3'
    else:
        return 'magnetic'

def ebs_price(n):
    if n > 100:
        return ebs_gp3_monthly_cost
    else:
        return ebs_magnetic_monthly_cost

In [ ]:
# Celda 31
df_on_rec['EBS_Type'] = df_on_rec['Iops_total_(R+W)-Max'].apply(ebs_type)
df_on_rec['EBS_Price'] = df_on_rec['Iops_total_(R+W)-Max'].apply(ebs_price)
# Calculamos el costo mensual por volumen
df_on_rec['Costo_Mensual_EBS'] = df_on_rec['EBS_Price'] * df_on_rec['Capacidad_Almacenamiento(GiB)']
# Calculamos el costo mensual por instancia por mes, a partir del costo por hora de cada instancia
df_on_rec['Costo_Mensual_EC2'] = df_on_rec['Precio_por_hr_EC2'] * 730

In [ ]:
# Celda 32
df_on_rec.sample(n=10)

## 3. Análisis Exploratorio de Datos

Con los datos recolectados, vamos a realizar una serie de gráficas que nos permitirán visualizar el tipo de instancias recomendadas, los sistemas operativos y la distribución de las versiones de Windows. 

Además, entregamos una lista con las máquinas con menor cantidad de IOPs (< 10), lo cual puede ser una indicación de VMs que están corriendo ociosamente on-prem, pero que probablemente no es necesario migrar a AWS.

In [ ]:
# Celda 33
# Con esta función vamos a graficar las columnas que sean de interés
def chart_category(data_column, width, height, add_percentage=False):
    plt.figure(figsize=(width,height))
    chart = sns.countplot(x=data_column, order = data_column.value_counts().index)
    chart.set_xticklabels(chart.get_xticklabels(),rotation=45)
    
    for p in chart.patches:
        total = len(data_column)
        if add_percentage == False:
            percentage = p.get_height()
        else:
            percentage = '{:.1f}%'.format(100 * p.get_height()/total) + '\n(' + str(p.get_height()) + ')'
        x = p.get_x() + p.get_width() / 2 
        y = p.get_y() + p.get_height() + 0.1
        chart.annotate(percentage, (x, y), size = 10)

In [ ]:
# Celda 34
df_on_rec.info()

In [ ]:
# Celda 35
# Vamos a graficar la distribución de instancias recomendadas. 
# Útil para entender los costos finales de EC2, al comparar diferentes escenarios
chart_category(df_on_rec['Instancia_EC2_Recomendada'], 20, 10, True)

In [ ]:
# Celda 36
# Otras Columnas interesantes
interesting_columns = ['Tipo_de_OS', 'EBS_Type']
for column in interesting_columns:
    chart_category(df_on_rec[column], 10, 5, True)

In [ ]:
# Celda 37
# Vamos a analizar también la distribución de los sistemas operativos windows. 
df_on_rec_win = df_on_rec[df_on_rec['Tipo_de_OS']=='Windows']
chart_category(df_on_rec_win['Operating_System'], 20, 10, True)

## VMs con 0 IOPs y con < 10 IOPs

En esta sección vamos a proporcionar las VMs que aparentemente no tienen ocupación, lo cual es una posibilidad de ahorrar costos al no considerarse dentro del proyecto de migración.

In [ ]:
# Celda 38
# Para tener consistencia con Migration Evaluator, vamos a llamar a las VMs con 0 IOPs como VMs zombies
df_zombie = df_on_rec[df_on_rec["Iops_total_(R+W)-Max"] == 0]

In [ ]:
# Celda 39
# Vamos a guardar las VMs Zombies en un CSV
df_zombie.to_csv('Zombie_vm.csv')

In [ ]:
# Celda 40
# Para tener consistencia con Migration Evaluator, vamos a llamar a las VMs con 0 IOPs como VMs zombies
df_zombie_ten_iops = df_on_rec[df_on_rec["Iops_total_(R+W)-Max"] < 10]

In [ ]:
# Celda 41
# Vamos a guardar las VMs Zombies en un CSV
df_zombie_ten_iops.to_csv('Zombie_vm_menos_10_iops.csv')

## Top 10 de la lista de instancias, por costo
Vamos a analizar cuáles son el Top 10 de las instancias, en términos de costos

In [ ]:
# Celda 42
ds_top10 = df_on_rec.sort_values('Costo_Mensual_EC2', ascending=False).head(10)

In [ ]:
# Celda 43
ds_top10.to_csv('top_10.csv', index=False)

# 4. Análisis de Costos y exportación de la tabla de recomendaciones

Al final, nos interesa saber cuál será el costo de la infraestructura (EC2, EBS y Data Transfer Out) sobre AWS. En esta sección proporcionamos un resumen de estos tres costos. Es recomendable evaluar distintos escenarios, por ejemplo, comparar el costo de una migración 1 a 1 (direct match) con una migración utilizando el 95 percentil, para determinar qué escenario nos conviene más.

## Costos EC2

In [ ]:
# Celda 44
ec2_cost = df_on_rec['Costo_Mensual_EC2'].sum().round(2)
print(f"El costo total mensual de EC2 es $ {ec2_cost} USD")

## Costos EBS

In [ ]:
# Celda 45
ebs_cost = df_on_rec['Costo_Mensual_EBS'].sum().round(2)
print(f"El costo total mensual de EBS es $ {ebs_cost} USD")

## Estimación DTO

In [ ]:
# Celda 46
total_gb_dto = df_on_rec['Data_Transfer_Out-GiB_escritos'].sum().round(2)*0.2

dto_monthly_cost = 0
if total_gb_dto < 10*1024:
    dto_monthly_cost = total_gb_dto * cost_10_tb
elif total_gb_dto < 50*1024:
    dto_monthly_cost = 10*1024*cost_10_tb + (total_gb_dto - 10*1024)*cost_40_tb
elif total_gb_dto < 150*1024:
    dto_monthly_cost = 10*1024*cost_10_tb + 40*1024*cost_40_tb + (total_gb_dto - 50*1024)*cost_100_tb
else:
    dto_monthly_cost = 10*1024*cost_10_tb + 40*1024*cost_40_tb + 100*1024*cost_100_tb + (total_gb_dto - 150*1024)*cost_150_tb
    
print(f"El costo total mensual de DTO es $ {dto_monthly_cost.round(2)} USD")

## Total Cost of Ownership (TCO)

In [ ]:
# Celda 47
print(f"El costo total de propiedad estimado es: $ {ec2_cost + ebs_cost + dto_monthly_cost.round(2)} USD (Mensuales)")

## Export del archivo de recomendaciones

In [ ]:
# Celda 48
# Escribiendo la tabla de recomendaciones para su análisis posterior
df_on_rec.to_csv(ec2_recommendation_file.split('.')[0] + "_output.csv", index=False)

# ------------ FIN